In [1]:
!pip install kaggle==1.5.6
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

     |████████████████████████████████| 58 kB 3.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72857 sha256=cf4e0bd7b30463425c8da3f46e700c8d10ac12be6f3409839f5ba3545fb00d72
  Stored in directory: /root/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [2]:
!kaggle competitions download -c 2021-ai-w4-p1

  0% 0.00/9.97k [00:00<?, ?B/s]
100% 9.97k/9.97k [00:00<00:00, 10.3MB/s]


In [3]:
!unzip 2021-ai-w4-p1.zip

Archive:  2021-ai-w4-p1.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [78]:
import pandas as pd
import numpy as np

train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
submit=pd.read_csv("submit.csv")

In [79]:
x=train.drop(["Diabetes"],axis=1)
y=train["Diabetes"]

In [80]:
 from sklearn.preprocessing import StandardScaler

transformer = StandardScaler().fit(x)
x_std=transformer.transform(x)
test_std=transformer.transform(test)

In [81]:
import torch

seed=777

torch.manual_seed(seed)
device='cuda' if torch.cuda.is_available() else 'cpu'

if device=='cuda':
  torch.cuda.manual_seed_all(seed)

In [82]:
x=np.array(x_std)
y=np.array(y)
test=np.array(test_std)

x=torch.FloatTensor(x)
y=torch.LongTensor(y)
test=torch.FloatTensor(test)

In [83]:
linear1=torch.nn.Linear(8,32)
linear2=torch.nn.Linear(32,32)
linear3=torch.nn.Linear(32,16)
linear4=torch.nn.Linear(16,2)

relu=torch.nn.ReLU()

dropout=torch.nn.Dropout(p=0.1)

In [84]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)

Parameter containing:
tensor([[-0.5360, -0.3451, -0.2453, -0.2763,  0.2040,  0.3958, -0.2544, -0.2888,
          0.3018, -0.1619, -0.3989, -0.3710,  0.4994, -0.4863, -0.1996,  0.0737],
        [ 0.4004, -0.2053,  0.5332,  0.0091,  0.1631,  0.5424,  0.5324, -0.1562,
         -0.3444,  0.1014,  0.4269,  0.5713,  0.2763, -0.1731,  0.0921,  0.4856]],
       requires_grad=True)

In [85]:
model=torch.nn.Sequential(linear1,relu,dropout,linear2,relu,dropout,linear3,relu,dropout,linear4).to(device)
optimizer = torch.optim.Adam(params= model.parameters(), lr=0.0001)
loss=torch.nn.CrossEntropyLoss().to(device)

In [86]:
model.train()
for stop in range(10000):
  optimizer.zero_grad()
  h=model(x.to(device))
  cost=loss(h,y.to(device))
  cost.backward()
  optimizer.step()
  if stop%1000==0:
    print(stop,cost.item())

0 0.9461566209793091
1000 0.45437660813331604
2000 0.4058244228363037
3000 0.3790278732776642
4000 0.3450919985771179
5000 0.3215785026550293
6000 0.3042454123497009
7000 0.2553637623786926
8000 0.26900628209114075
9000 0.2468721717596054


In [87]:
pred=list()
with torch.no_grad():
  model.eval()
  test_y=model(test.to(device))
  a,b=torch.max(test_y,dim=1)
  pred.extend(b.tolist())

In [88]:
submit["Diabetes"]=pred
submit.to_csv("submit.csv",index=False)

In [89]:
submit

,ID,Diabetes
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
226,226,0
227,227,0
228,228,0
229,229,0


In [90]:
!kaggle competitions submit -c 2021-ai-w4-p1 -f submit.csv -m "Message"

100% 1.26k/1.26k [00:00<00:00, 7.25kB/s]
Successfully submitted to 당뇨병 예측